# Proceso de Curación

Se pretende construir un modelo predictivo de la calidad del aire en las principales ciudades de España a partir de la meteorología del lugar para tratar de determinar el efecto del confinamiento y las restricciones debidas a la pandemia del COVID-19.

In [4]:
suppressMessages(library(tidyverse))

# repository directory
setwd("~/Repositories/AirQualityCOVID/")

Para ello se han de obtener datos tanto de calidad del aire como meteorológicos suficientes para contruir del modelo predictivo. Se ha decidido utilizar el periodo de tiempo desde el `1-Enero-2013` al `31-Diciembre-2019` para entrenar el modelo.

Se ha escogido tomar datos desde el $2013$ para la creación del modelo debido a que es el año a partir del cual hay datos en el portal de la [_European Enviroment Aency_ (**EEA**)](https://discomap.eea.europa.eu/map/fme/AirQualityExport.htm)

In [2]:
suppressMessages(library(lubridate))

start_dt <- ymd("2013-01-01")
end_dt <- ymd("2020-12-30")

---

## Calidad del Aire

> _El proceso de curación completo de los datos de calidad del aire se ha realizado mediante el script `src/Curation/airQuality.R`._

Las restricciones de movilidad debidas al confinamiento por el COVID-19 deberían haber supuesto una disminución en el uso de vehículos, lo cual repercutiría directamente en los niveles de concentración de los contaminantes producidos por los vehículos. 

Por ello en este estudio nos hemos centrado en las estaciones de calidad del aire de tipo **tráfico urbano** situadas en ciudades españolas con más de **> 100 000 habitantes**.

Los contaminantes escogidos han sido: 
* _Óxido de Nitrógeno_ ($NO$) $\rightarrow$ **no**
* _Dióxido de Nitrógeno_ ($NO_2$) $\rightarrow$ **no2**
* _Ozono_ ($O_3$) $\rightarrow$ **o3**
* _Partículas de Materia de diámetro $\leq 10 \mu m$_ ($PM10$) $\rightarrow$ **pm10**
* _Partículas de Materia de diámetro $\leq 2.5 \mu m$_ ($PM2.5$) $\rightarrow$ **pm2.5**

In [3]:
site_type <- "traffic"
site_area <- "urban"

pollutants <- c("no", "no2", "o3", "pm10", "pm2.5")

En el archivo `data/xlsx/estaciones-CA-JA.xlsx` se encuentra información sobre las estaciones de calidad del aire de las ciudades españolas de $> 100 000$ habitantes.

In [4]:
suppressMessages(library(openxlsx))

# AQ station in cities with more than 100000 inhabitants
sites.100mil <- read.xlsx("data/xlsx/estaciones-CA-JA.xlsx",
                          sheet="ciudades-100000-A") %>% 
                    select("Municipio", "Población",
                           "Estación.tráfico", "Código.estación") 

Para la obtención de los datos de calidad del aire se ha utilizado el paquete [`saqgetr`](https://github.com/skgrange/saqgetr), el cual utiliza como principal fuente de datos la **EEA**. 

> Este paquete permite obtener la concentración de los contaminantes en $\mu g \cdot m^{-3}$.

Por ello se utiliza dicho paquete para comprobar que la información del archivo anterior concuerda con la disponible por la **EEA** y las estaciones cumplen con los requisitos mínimos descritos anteriormente.

In [8]:
suppressMessages(library(saqgetr))

spain.sites <- get_saq_sites() %>%
    filter(country == "spain",
           site %in% sites.100mil$"Código.estación",
           site_type == site_type,
           site_area == site_area,
           date_start <= start_dt,
           ) %>%
    select(site, site_name, latitude, longitude, elevation, 
           country, site_type, site_area, date_start, date_end)

sites.AQ <- merge(x = spain.sites,
                  y = sites.100mil,
                  by.x = "site", by.y="Código.estación",
                  all.x = TRUE) 
head(sites.AQ)

,site,site_name,latitude,longitude,elevation,country,site_type,site_area,date_start,date_end,Municipio,Población,Estación.tráfico
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<dttm>,<dttm>,<chr>,<dbl>,<chr>
1,es0041a,DIRECCIÓN DE SALUD,43.25883,-2.945650,32,spain,traffic,urban,1986-04-01,2021-04-10 23:00:00,Bilbao,346843,María Diaz de Haro
2,es0110a,ERANDIO,43.30268,-2.977240,4,spain,traffic,urban,1997-01-01,2021-04-10 23:00:00,Bilbao,24350,Erandio
3,es0115a,PLAZA DE ESPAÑA,40.42417,-3.712222,637,spain,traffic,urban,1986-01-01,2021-04-10 23:00:00,Madrid,3266127,PLAZA DE ESPAÑA
4,es0118a,ESCUELAS AGUIRRE,40.42167,-3.682222,672,spain,traffic,urban,2002-11-19,2021-04-10 23:00:00,Madrid,3266128,ESCUELAS AGUIRRE
5,es0120a,RAMÓN Y CAJAL,40.45167,-3.677222,708,spain,traffic,urban,2002-01-01,2021-04-10 23:00:00,Madrid,3266129,RAMÓN Y CAJAL
6,es0817a,LA RANILLA,37.38425,-5.959620,29,spain,traffic,urban,1997-01-01,2021-04-11 00:00:00,Sevilla,688593,LA RANILLA


Sin embargo, esta información solo hace referencia a los datos generales de la estación, sin indicar si existen datos suficientes para el estudio de cada contaminante del estudio. Esto se debe comprobar de forma individual para cada contaminante una vez descargado los datos.

```R
    data.AQ <- get_saq_observations(
        site = "es0000a"
        variable = "XO",
        start = start_dt,
        end = end_dt,
        valid_only = TRUE,
        verbose = FALSE
    )
```

Para cada contaminante de estudio de cada estación se obtienen los siguientes valores:

* **start_yr**: Fecha de comienzo de toma de datos. El valor mínimo posible es `start_dt`.
* **end_yr**: Fecha de fin de toma de datos. El valor máximo posible es `end_dt`.
* **hv.min**: `Booleano` si hay datos con resolución diaria suficientes ($> 80 \%$) durante el periodo de mayor interes del estudio.
    > Se toma el intervalo de tiempo `1-Marzo-2020` al `30-Junio-2020` como el periodo de mayor interes del estudio ya que comprende los meses desde el comienzo del primer estado de alarma hasta la vuelta a la primera "normalidad" (fin de las fases de la desescalada) 
* **mss.wk**: Número de valores nulos con resolución semanal. Número de semanas faltantes.
* **mss.mnth**: Número de valores nulos con resolución mensual. Número de meses faltantes.
* **mss.yr**: Número de valores nulos con resolución anual. Número de años faltantes.

A partir de estos valores se han seleccionado aquellos contaminantes en cada estación con:
```R
    hv.min == TRUE
    mss.yr < 5
```

In [6]:
valid.df <- read.csv("data/Curation/checked_AQ.csv")
head(valid.df)

,site,Pollutant,site_name,latitude,longitude,elevation,country,site_type,site_area,date_start,date_end,Municipio,Población,Estación.tráfico
,<chr>,<chr>,<chr>,<dbl>,<dbl>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<chr>
1,es0041a,no2,DIRECCIÓN DE SALUD,43.25883,-2.94565,32,spain,traffic,urban,1986-04-01 00:00:00,2021-04-10 23:00:00,Bilbao,346843,María Diaz de Haro
2,es0041a,pm10,DIRECCIÓN DE SALUD,43.25883,-2.94565,32,spain,traffic,urban,1986-04-01 00:00:00,2021-04-10 23:00:00,Bilbao,346843,María Diaz de Haro
3,es0110a,no,ERANDIO,43.30268,-2.97724,4,spain,traffic,urban,1997-01-01 00:00:00,2021-04-10 23:00:00,Bilbao,24350,Erandio
4,es0110a,no2,ERANDIO,43.30268,-2.97724,4,spain,traffic,urban,1997-01-01 00:00:00,2021-04-10 23:00:00,Bilbao,24350,Erandio
5,es0110a,pm10,ERANDIO,43.30268,-2.97724,4,spain,traffic,urban,1997-01-01 00:00:00,2021-04-10 23:00:00,Bilbao,24350,Erandio
6,es0110a,pm2.5,ERANDIO,43.30268,-2.97724,4,spain,traffic,urban,1997-01-01 00:00:00,2021-04-10 23:00:00,Bilbao,24350,Erandio


---

## Meteorológicos

Una vez seleccionadas las estaciones de calidad del aire se ha procedido a obtener los datos meteorológicos correspondientes a cada estación, que servirán de predictores para el modelo, seleccionando aquellas variables que se podían considerar más influyentes en los niveles de contaminación de los contaminantes. 

Sin embargo, no se ha podido encontrar una base de datos completa y válida con datos suficientes de todas las variables meteorológicas que se querían usar para el estudio. Por ello, se han utilizado tres fuentes distintas de datos meteorológicos, siendo la fuente principal la _[Agencia Estatal de Meteorología (**AEMET**)](http://www.aemet.es/es/portada)_. A estos datos se les ha añadido datos de dirección y velocidad del viento de _[National Oceanic and Atmospheric Administration Integrated Surface Database (**NOAA ISD**)](https://www.ncdc.noaa.gov/isd)_ y datos de la humedad relativa y la radiación de [_ERA5-Land_](https://cds.climate.copernicus.eu/cdsapp#!/search?type=dataset&text=era5-land)

Para todas las fuentes se han utilizado datos de los puntos más cercanos a las estaciones de calidad del aire seleccionadas anteriormente.

### Fuente: AEMET

> La curación de los datos meteorológicos obtenidos de la **AEMET** se ha realizado utilizando el módulo `src/Curation/meteo`

Para la obtención de los datos meteorológicos de la **AEMET** se ha utilizado la API REST [AEMET OpenData](https://opendata.aemet.es/centrodedescargas/inicio) desarrollada para ello. Para facilitar la interacción con la API se ha desarrollado una clase en Python `DownloadAEMET` que permite obtener los datos meteorológicos de cada estación en las fechas deseadas en formato `pandas.DataFrame`. Se le ha incluido además un método para la obtención de los datos meteorológicos de las `n` estaciones más cercanas a una posición dada por su latitud y longitud, calculando la distancia de dichas estaciones al punto. La clase `DownloadAEMET` se encuentra en el script `src/Curation/meteo/AEMET.py` dentro del modulo `meteo`.

Para convertir los archivos _json_ facilitados por la API de AEMET en valores útiles para el estudio ha sido necesario realizar algunas transformaciones:

* _**Coordenadas**_: Tanto la latitud como la longitud están dadas en grados, minutos y segundos; indicando la dirección norte **N**, sur **S**, este **E** u oeste **W**. Estos valores se han traducido a valores de grados con decimales, indicando su dirección mediante el signo positivo o negativo.
    > $$\begin{matrix}
Latitud & Londgitud \\ 
\left.\begin{matrix}
        \textrm{Este} & \textbf{E} & \rightarrow & + & \\
        \textrm{Norte} & \textbf{N} & \rightarrow & + &
    \end{matrix}\right| & 
\begin{matrix}
        \textrm{Oeste} & \textbf{W} & \rightarrow & - \\
        \textrm{Sur} & \textbf{S} & \rightarrow & -
    \end{matrix}
\end{matrix}$$
    
* _**Decimales**_: Cambiar el separador decimal de coma decimal a punto decimal 
* _**Precipitación**_: Dentro de la variable **prec** se incluye el string `Ip` para denotar precipitaciones menores de 0.1 mm. Para convertirlo en valor numérico se le ha asignado el valor `prec = 0.5 mm`.
* _**Horas Varias**_: Dentro de las variables horarias se incluye el string `varias` para denotar que dicho valor al que hace referencia se alcanzó a varias horas del día. Este valor se ha sustituído por el valor numérico `-2`. No obstante, estas variables horarias no se utilizarán en el estudio.

Los datos obtenidos de **AEMET** se encuentran con resolución diaria por lo que ésta será la resolución temporal mínima que se utilizará en el estudio. Además, de todas las variables meteorológicas que facilita **AEMET** se han descartado las variables horarias (Horas a las que se alcanza un valor u horas de sol) y las variables relacionadas con las rachas de viento. A continuación se muestran las variables seleccionadas para el estudio.

| Variable |                     Descripción                      | Unidad |
|:--------:|:----------------------------------------------------:|:------:|
|   fecha  |              fecha del dia (AAAA-MM-DD)              |   -    |
|   tmed   |               Temperatura media diaria               |   ºC   |
|   prec   |           Precipitación diaria de 07 a 07            |   mm   |
|   tmax   |              Temperatura Mínima del día              |   ºC   |
|   tmin   |              Temperatura Máxima del día              |   ºC   |
| presmax  | Presión máxima al nivel de referencia de la estación |  hPa   |
| presmin  | Presión mínima al nivel de referencia de la estación |  hPa   |


A cada estación de calidad del aire del estudio se le ha asignado la estación de la AEMET más cercana con al menos el $80 \%$ `min_prop=0.8` de los datos de las variables seleccionadas `selected_cl` de entre las 10 estaciones más cercanas `n=10`.

```python

# Initialize AEMET API class with the api key
Aemet = DownloadAEMET(apikey=apikey)

download_nearest_data(aemet=Aemet, 
                      siteAQ=pd.DataFrame({"site": "es0000a",
                                           "latitude": "42.38",
                                           "longitude": "-3.8"
                                          }),
                      stdy_prd=[start_date, end_date],
                      remove_cl=["sol", "horatmin", "horatmax", "horaracha",
                                 "dir", "velmedia", "horaPresMax", "horaPresMin"],
                      selected_cl=["fecha", "tmed", "prec",
                                   "tmin", "tmax", "presMax", "presMin"],
                      n=10,
                      min_prop=0.8
                     )
```

In [2]:
aemet <- read.csv("data/Curation/checked_AEMET.csv")
head(aemet)

,latitud,provincia,altitud,indicativo,nombre,indsinop,longitud,dist,siteAQ
,<dbl>,<chr>,<int>,<chr>,<chr>,<int>,<dbl>,<dbl>,<chr>
1,43.29806,BIZKAIA,42,1082,BILBAO AEROPUERTO,8025,-2.906389,5.396849,es0041a
2,43.29806,BIZKAIA,42,1082,BILBAO AEROPUERTO,8025,-2.906389,5.756578,es0110a
3,40.41194,MADRID,667,3195,"MADRID, RETIRO",8222,-3.678056,1.137154,es0118a
4,40.41194,MADRID,667,3195,"MADRID, RETIRO",8222,-3.678056,4.417472,es0120a
5,37.41667,SEVILLA,34,5783,SEVILLA AEROPUERTO,8391,-5.879167,7.968651,es0817a
6,37.41667,SEVILLA,34,5783,SEVILLA AEROPUERTO,8391,-5.879167,11.220392,es0890a


### Fuente: NOAA

> La curación de los datos meteorológicos obtenidos de **NOAA ISD** se ha realizado utilizando el script `src/Curation/worldMet.R`

[`worldmet`](https://github.com/davidcarslaw/worldmet)

```R

getMeta(lat = sites.AQ[sites.AQ\$ site == st, ]\$latitude[1],
        lon = sites.AQ[sites.AQ\$ site == st, ]\$longitude[1],
        end.year = "current",
        n = 10, returnMap = F)
```

In [3]:
worldmet <- read.csv("data/Curation/checked_WorldMet.csv")
head(worldmet)

,usaf,wban,station,ctry,st,call,latitude,longitude,elev.m.,begin,end,code,longr,latr,dist,siteAQ
,<int>,<int>,<chr>,<chr>,<lgl>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>
1,80250,99999,BILBAO,SP,NA,LEBB,43.301,-2.911,42.1,1973-01-01,2021-03-06,080250-99999,-0.05080653,0.7557450,5.464009,es0041a
2,80250,99999,BILBAO,SP,NA,LEBB,43.301,-2.911,42.1,1973-01-01,2021-03-06,080250-99999,-0.05080653,0.7557450,5.363540,es0110a
3,82230,99999,CUATRO VIENTOS,SP,NA,LEVS,40.371,-3.785,691.0,1973-07-19,2021-03-06,082230-99999,-0.06606071,0.7046069,10.367903,es0118a
4,82210,99999,BARAJAS,SP,NA,LEMD,40.494,-3.567,609.6,1931-01-02,2021-03-06,082210-99999,-0.06225589,0.7067536,10.444349,es0120a
5,83910,99999,SEVILLA,SP,NA,LEZL,37.418,-5.893,33.8,1973-01-01,2021-03-06,083910-99999,-0.10285225,0.6530673,6.979568,es0817a
6,83910,99999,SEVILLA,SP,NA,LEZL,37.418,-5.893,33.8,1973-01-01,2021-03-06,083910-99999,-0.10285225,0.6530673,10.069231,es0890a


### Fuente: ERA5-Land